# Import

In [162]:
#packages import
!pip install kaggle
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
os.environ['KAGGLE_USERNAME'] = "sabrinaauger"
os.environ['KAGGLE_KEY'] = "8db79b4ccae98a2c74af940e1f1b7c35"
import kaggle

# Import the dataset from Kaggle

In [163]:
#Retrieve the data from Kaggle platform
!kaggle datasets download -d zynicide/wine-reviews

wine-reviews.zip: Skipping, found more recently modified local copy (use --force to force download)


In [164]:
#Unzip the data zip file
!unzip wine-reviews.zip

Archive:  wine-reviews.zip
replace winemag-data-130k-v2.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [512]:
#Retrieve the data from csv unzipped files into two dataframes
wine1_df=pd.read_csv("/Users/sabrinaauger/code/sabrinaauger/wino/raw_data/winemag-data-130k-v2.csv")

In [513]:
wine2_df=pd.read_csv("/Users/sabrinaauger/code/sabrinaauger/wino/raw_data/winemag-data_first150k.csv")

In [482]:
#wine2_df.head()

In [507]:
#wine1_df.head()

# Data vizualisation

In [600]:
#Columns overview of both datasets
wine1_df.columns, wine2_df.columns

(Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
        'price', 'province', 'region_1', 'region_2', 'taster_name',
        'taster_twitter_handle', 'title', 'variety', 'winery'],
       dtype='object'),
 Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
        'price', 'province', 'region_1', 'region_2', 'variety', 'winery'],
       dtype='object'))

In [601]:
#Analysing the variety column and the unique value
#wine1_df['variety'].unique()

In [602]:
# Looking at the description of 1 specific wine
index_of_record_to_view = 3
des = wine2_df.loc[index_of_record_to_view, 'description']
des

"This spent 20 months in 30% new French oak, and incorporates fruit from Ponzi's Aurora, Abetina and Madrona vineyards, among others. Aromatic, dense and toasty, it deftly blends aromas and flavors of toast, cigar box, blackberry, black cherry, coffee and graphite. Tannins are polished to a fine sheen, and frame a finish loaded with dark chocolate and espresso. Drink now through 2032."

In [603]:
wine_review1_df.shape, wine_review2_df.shape

((120915, 10), (120915, 10))

# Preprocessing the data 

## Cleaning wine1_df and wine2_df dataset 

### Removing Unuseful columns for the recommendation system

In [604]:
#Remove both columns taster name and taster twitter_handle as they are not relevant for our analysis
wine_review1_df=wine1_df.drop(columns=["taster_twitter_handle","taster_name","Unnamed: 0"])
wine_review2_df=wine2_df.drop(columns=["Unnamed: 0"])
#wine_review2_df.head()


In [605]:
#Extracting the year from the column title in wine1_df

# Extract the year from title into a dedicated column using regular expression
#regular instruction (\d{4}) simply looks for four consecutive digits in the strings
wine_review1_df['Year']=wine_review1_df['title'].str.extract(r'(\d{4})')

missing_year=wine_review1_df['Year'].isna().sum()
print(f'Missing data in year variable : {missing_year} - Decision taken: not adding this additional variable')
wine_review1_df.drop(columns='Year', inplace=True)

Missing data in year variable : 4609 - Decision taken: not adding this additional variable


### Duplicates

In [606]:
#checking if there is any overall duplicates in both datasets
print(wine_review1_df.duplicated().sum())
print(wine_review2_df.duplicated().sum())
print("Note - TODO: Check the duplicates 'title' after cleaning the missing data")

9983
53079
Note - TODO: Check the duplicates 'title' after cleaning the missing data


In [607]:
### Checking at the duplicate
duplicates=wine_review1_df[wine_review1_df.duplicated(subset=('title'))]
duplicates_sorted = duplicates.sort_values(by=['title'])
#duplicates_sorted
#should not be considered as duplicates - cleaning the data first


duplicates=wine_review2_df[wine_review2_df.duplicated(subset=('designation'))]
duplicates_sorted = duplicates.sort_values(by=['designation'])
#duplicates_sorted
#need to clean the data to interpretate the duplicates

### Missing data 

In [608]:
#Overall missing data in wine_review1_df dataset
wine_review1_df.isna().sum(),wine_review2_df.isna().sum()

(country           63
 description        0
 designation    37465
 points             0
 price           8996
 province          63
 region_1       21247
 region_2       79460
 title              0
 variety            1
 winery             0
 dtype: int64,
 country            5
 description        0
 designation    45735
 points             0
 price          13695
 province           5
 region_1       25060
 region_2       89977
 variety            0
 winery             0
 dtype: int64)

#### Variety missing data cleaning : 

In [609]:
#Variety missing data 
wine_review1_df[wine_review1_df['variety'].isna()]
#Only one record is missing in wine_review1_df and most of the data for this wine is missing - Decided to remove the record
wine_review1_df = wine_review1_df.dropna(subset=['variety'])

#As title contains already the information of designation column - decision : drop designation 
#wine_review1_df=wine_review1_df.drop(columns=['designation'])

#print('Decision taken: Dropped the only record wine with missing variety as the record had only missing data')
#print('Decision taken: Dropped column designation as title contains already the information')

#### Designation missing data cleaning

In [610]:
m1=wine_review1_df['designation'].isna().sum()
m2=wine_review2_df['designation'].isna().sum()
print(f'Designation missing in wine_review1_df:{m1},in wine_review2_df:{m2}')

Designation missing in wine_review1_df:37464,in wine_review2_df:45735


##### Wine_review1_df

In [611]:
#Filling the missing designation using title variable 

wine_review1_df['designation'].fillna(wine_review1_df['title'], inplace=True)
def remove_year(designation):
    pattern = r'\b\d{4}\b'  # Regex pattern to match four-digit year
    return re.sub(pattern, '', designation).strip()

# Remove year from 'Designation' column
wine_review1_df['designation'] = wine_review1_df['designation'].apply(remove_year)

#Dealing with the parentheses values in designation
def remove_parentheses_text(designation):
    pattern = r'\s*\([^)]*\)'  # Regex pattern to match text within parentheses
    return re.sub(pattern, '', designation).strip()

# Remove text within parentheses from 'Designation' column
wine_review1_df['designation'] = wine_review1_df['designation'].apply(remove_parentheses_text)

##### Wine_review2_df

In [613]:
#As no title provide the designation of the wine the missing designation in wine_review2_df must be dropped 
wine_review2_df.dropna(subset='designation', inplace=True)


In [612]:
wine_review2_df.isna().sum()

country            5
description        0
designation    45735
points             0
price          13695
province           5
region_1       25060
region_2       89977
variety            0
winery             0
dtype: int64

#### Region missing data cleaning
Analysing the region_1 and region_2 variables and missing data:
- 1st strategy: Creating a unique new column 'region' retrieving the information from either region1 or region2 As 'region1' and 'region2' provide and contain the same information - remaining 21k missing data - NOT APPLIED
- 2nd strategy: Understanding the value of the region_1 and region_2 into our analysis
- => Decision taken: Dropping the columns region_1 and region_2 as they don;t provide any addiitonal information than the other variables


##### Wine_review1_df

In [542]:
wine_review1_df.head()

,country,description,designation,points,price,province,region_1,region_2,title,variety,winery
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,US,"Tart and snappy, the flavors of lime flesh and...",Rainstorm Pinot Gris,87,14.0,Oregon,Willamette Valley,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [614]:
#Analysing the region_1 and region_2 variables 
#1st strategy: region_1 and region_2 Filling the missing data in region_1 with region_2 when applicable
wine_review1_df['region_1'].fillna(wine_review1_df['region_2'], inplace=True)
wine_review1_df['region_1'].isna().sum()


21246

In [615]:
#Removing the region_2 column from the dataset
wine_review1_df.drop(columns='region_2', inplace=True)

In [617]:
#Removing the Un-named created column
#wine_review1_df.drop(columns='Unnamed: 0', inplace=True)

In [618]:
wine_review1_df.isna().sum()

country           63
description        0
designation        0
points             0
price           8996
province          63
region_1       21246
title              0
variety            0
winery             0
dtype: int64

In [619]:
import re #import re module for regular expressions 
#filling the missing region with the region displayed in the title

#Function to extract the content within parentheses from title column 
def extract(s):
    if isinstance(s, str):
        extracted = re.search(r'\((.*?)\)', s)
        if extracted:
            return extracted.group(1)
    return None
wine_review1_df['region_1'].fillna(wine_review1_df['title'].apply(extract), inplace=True)

wine_review1_df[wine_review1_df['region_1'].isna()]

#412 instances don't have a region displayed but have other useful data available


,country,description,designation,points,price,province,region_1,title,variety,winery
261,Argentina,"A blend of 50% Malbec with Cabernet, Syrah and...",Estate Blend Gran Reserva,89,37.0,Mendoza Province,None,Algodon 2008 Estate Blend Gran Reserva Red,Red Blend,Algodon
594,France,This medium-bodied blend of 50% Grenache and 5...,La Decelle,89,20.0,Rhône Valley,None,Lavau 2015 La Decelle Red,Rhône-style Red Blend,Lavau
622,US,"Mostly Syrah, with just a dash of Petite Sirah...",Sidewinder,87,25.0,Idaho,None,Coiled 2009 Sidewinder Red,Red Blend,Coiled
623,US,"This unusual blend of 85% Syrah, 10% Petit Ver...",Black Mamba,87,28.0,Idaho,None,Coiled 2010 Black Mamba Red,Red Blend,Coiled
913,NaN,"Amber in color, this wine has aromas of peach ...",Asureti Valley,87,30.0,NaN,None,Gotsa Family Wines 2014 Asureti Valley Chinuri,Chinuri,Gotsa Family Wines
...,...,...,...,...,...,...,...,...,...,...
129297,Spain,Jammy black-fruit aromas push up against raisi...,Payoya Negra,90,33.0,Andalucia,None,Finca la Melonera 2014 Payoya Negra Red,Red Blend,Finca la Melonera
129407,NaN,Dry spicy aromas of dusty plum and tomato add ...,Reserve,89,22.0,NaN,None,El Capricho 2015 Reserve Cabernet Sauvignon,Cabernet Sauvignon,El Capricho
129408,NaN,El Capricho is one of Uruguay's more consisten...,Reserve,89,22.0,NaN,None,El Capricho 2015 Reserve Tempranillo,Tempranillo,El Capricho
129590,NaN,"A blend of 60% Syrah, 30% Cabernet Sauvignon a...",Shah,90,30.0,NaN,None,Büyülübağ 2012 Shah Red,Red Blend,Büyülübağ


- Filling the missing region_1 data when applicable using title

In [620]:
#checking the data that have both missing province and region
check = wine_review1_df['region_1'].isnull() & wine_review1_df['province'].isnull()
# Filter the DataFrame based on the condition
rows_missing_region_province = wine_review1_df[check]
#rows_missing_region_province

wine_review1_df['region_1'].fillna(wine_review1_df['province'], inplace=True)


In [621]:
wine_review1_df['region_1'].isna().sum()

63

In [622]:
#wine_review1_df[wine_review1_df['region_1'].isna()]
#dropping the 63 remaining instances that don't have country , province and region filled
wine_review1_df.dropna(subset='region_1', inplace=True)

##### Wine_review2_df

In [623]:
#Analysing the region_1 and region_2 variables 
#1st strategy: region_1 and region_2 Filling the missing data in region_1 with region_2 when applicable
wine_review2_df['region_1'].fillna(wine_review2_df['region_2'], inplace=True)
wine_review2_df['region_1'].isna().sum()


19581

In [624]:
#Removing the region_2 column from the dataset
wine_review2_df.drop(columns='region_2', inplace=True)


In [625]:

wine_review2_df['region_1'].fillna(wine_review2_df['province'], inplace=True)
wine_review2_df[wine_review2_df['region_1'].isna()]
wine_review2_df.dropna(subset='region_1', inplace=True)

### Missing province variable

##### Wine_review1_df

In [626]:
#Displaying the missing data
wine_review1_df[wine_review1_df['province'].isna()]
wine_review1_df.shape[0]

129907

In [627]:
#Trying to fill the missing province with the title information 

def extract_province_from_title(title):
    pattern = r'\((.*?)\)'  # Regex pattern to match text within parentheses
    match = re.search(pattern, title)
    if match:
        return match.group(1)
    else:
        return None

# Extract province from 'Title' column
wine_review1_df['extracted_Province'] = wine_review1_df['title'].apply(extract_province_from_title)

# Fill missing 'Province' values with extracted province information
wine_review1_df['province'].fillna(wine_review1_df['extracted_Province'], inplace=True)
wine_review1_df.drop(columns=['extracted_Province'], inplace=True)

In [628]:
wine_review1_df.isna().sum()

country           0
description       0
designation       0
points            0
price          8992
province          0
region_1          0
title             0
variety           0
winery            0
dtype: int64

##### Wine_review2_df

In [629]:
wine_review2_df.isna().sum()

country            0
description        0
designation        0
points             0
price          10271
province           0
region_1           0
variety            0
winery             0
dtype: int64

### Missing price
Decision taken : Missing price filled with mean price of wines with same variety

##### Wine_review1_df

In [630]:
#Frequency of each unique variety
wine_variety=pd.DataFrame(wine_review1_df['variety'].value_counts())

#Calculating the mean price per variety
mean_price_per_variety=pd.DataFrame(wine_review1_df.groupby('variety')['price'].mean())
mean_price_per_variety

#Replacing the missing price with mean price of same variety of wine when applicable 

wine_review1_df = pd.merge(wine_review1_df, mean_price_per_variety, on='variety', how='left',suffixes=('', '_mean'))
wine_review1_df['price'] = wine_review1_df.apply(lambda row: row['price_mean'] if pd.isnull(row['price']) else row['price'], axis=1)


In [631]:
wine_review1_df=wine_review1_df.drop(columns=['price_mean'])

In [632]:
wine_review1_df['price']=wine_review1_df['price'].round(2)
#wine_review1_df

In [633]:
wine_review1_df=wine_review1_df.dropna()
wine_review1_df.isna().sum()

country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
title          0
variety        0
winery         0
dtype: int64

##### Wine_review2_df

In [634]:
#Frequency of each unique variety
wine_variety=pd.DataFrame(wine_review2_df['variety'].value_counts())

#Calculating the mean price per variety
mean_price_per_variety=pd.DataFrame(wine_review2_df.groupby('variety')['price'].mean())
mean_price_per_variety

#Replacing the missing price with mean price of same variety of wine when applicable 

wine_review2_df = pd.merge(wine_review2_df, mean_price_per_variety, on='variety', how='left',suffixes=('', '_mean'))
wine_review2_df['price'] = wine_review2_df.apply(lambda row: row['price_mean'] if pd.isnull(row['price']) else row['price'], axis=1)


In [635]:
wine_review2_df=wine_review2_df.drop(columns=['price_mean'])

In [636]:
wine_review2_df['price']=wine_review2_df['price'].round(2)
#wine_review1_df

In [637]:
wine_review2_df=wine_review2_df.dropna()
wine_review2_df.isna().sum()

country        0
description    0
designation    0
points         0
price          0
province       0
region_1       0
variety        0
winery         0
dtype: int64

In [638]:
wine_review1_df.columns, wine_review2_df.columns

(Index(['country', 'description', 'designation', 'points', 'price', 'province',
        'region_1', 'title', 'variety', 'winery'],
       dtype='object'),
 Index(['country', 'description', 'designation', 'points', 'price', 'province',
        'region_1', 'variety', 'winery'],
       dtype='object'))

In [599]:
wine_review2_df

,country,description,designation,points,price,province,region_1,title,variety,winery
0,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,Douro,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
1,US,"Tart and snappy, the flavors of lime flesh and...",Rainstorm Pinot Gris,87,14.0,Oregon,Willamette Valley,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
2,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
3,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks
4,Spain,Blackberry and raspberry aromas show a typical...,Ars In Vitro,87,15.0,Northern Spain,Navarra,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,Tandem
...,...,...,...,...,...,...,...,...,...,...
120910,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,Erben Müller-Burggraef,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef)
120911,US,Citation is given as much as a decade of bottl...,Citation Pinot Noir,90,75.0,Oregon,Oregon,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation
120912,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser
120913,France,"A dry style of Pinot Gris, this is crisp with ...",Domaine Marcel Deiss Pinot Gris,90,32.0,Alsace,Alsace,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss
